# Système d'analyse faciale pour l'entrée dans l'enceinte de l'école Albert School

# Découverte de la FaceNet Pytorch

####  Installation

In [3]:
pip install facenet-pytorch

  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pillow-10.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached torch-2.2.2-cp312-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached facenet_pytorch-2.6.0-py3-none-any.whl (1.9 MB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
Using cached pill

In [5]:
pip install --upgrade jupyter

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyter_console-6.6.3-py3-none-any.whl (24 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [7]:
from facenet_pytorch import MTCNN, InceptionResnetV1

In [10]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()

100%|████████████████████████████████████████| 107M/107M [01:25<00:00, 1.31MB/s]


In [11]:
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [17]:
mtcnn = MTCNN(image_size=16, margin=2)

### Transformation de l'image RGBA en RGB

Les images sont en 4 dimensions (RVBA - transparence inclu), or le modèle ne traite que les images en 3 dimensions (RVB),
d'où la nécessité de passer par la transformation suivante :

In [19]:
from PIL import Image
from facenet_pytorch import MTCNN

# Charger l'image
img = Image.open('photocv.png')

# Convertir l'image en RGB si elle est en RGBA
if img.mode == 'RGBA':
    img = img.convert('RGB')

# Créer l'instance MTCNN
mtcnn = MTCNN()

# Détecter et recadrer le visage
img_cropped = mtcnn(img, save_path='photocv_cropped.png')

# Calculer l'embedding
img_embedding = resnet(img_cropped.unsqueeze(0))

In [21]:
from PIL import Image

img = Image.open('photocv_cropped.png')

# Get cropped and prewhitened image tensor
img_cropped = mtcnn(img, save_path='photocv_cropped.png')

# Calculate embedding (unsqueeze to add batch dimension)
img_embedding = resnet(img_cropped.unsqueeze(0))

# Or, if using for VGGFace2 classification
resnet.classify = True
img_probs = resnet(img_cropped.unsqueeze(0))

### COMPARER DEUX VISAGES IDENTIQUES

In [42]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch

# Initialisation de MTCNN pour la détection faciale et InceptionResnetV1 pour les embeddings
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def compare_images(img_path_1, img_path_2, threshold=0.8):

    # Charger les deux images
    img1 = Image.open('yasmine_cropped.png')
    img2 = Image.open('yasmine_cropped.png')
    
    # Détecter et recadrer les visages dans les deux images
    img1_cropped = mtcnn(img1)
    img2_cropped = mtcnn(img2)
    
    # Si un visage n'a pas été détecté, renvoyer une erreur
    if img1_cropped is None or img2_cropped is None:
        print("Erreur : visage non détecté dans l'une des images.")
        return False
    
    # Calculer les embeddings des deux images
    img1_embedding = resnet(img1_cropped.unsqueeze(0))
    img2_embedding = resnet(img2_cropped.unsqueeze(0))
    
    # Calculer la distance entre les deux embeddings (distance Euclidienne)
    distance = torch.dist(img1_embedding, img2_embedding).item()
    
    # Comparer la distance avec le seuil
    if distance < threshold:
        print(f"Les visages correspondent avec une distance de {distance:.2f}.")
        return True
    else:
        print(f"Les visages ne correspondent pas, distance : {distance:.2f}.")
        return False

# Exemple d'utilisation
img_captured = 'yasmine_cropped.png'
img_reference = 'yasmine_cropped.png'
result = compare_images(img_captured, img_reference)


Les visages correspondent avec une distance de 0.00.


### COMPARER DEUX VISAGES NON IDENTIQUES

In [38]:
from PIL import Image
from facenet_pytorch import MTCNN

# Charger l'image
img = Image.open('hoda.jpeg')

# Convertir l'image en RGB si elle est en RGBA
if img.mode == 'RGBA':
    img = img.convert('RGB')

# Créer l'instance MTCNN
mtcnn = MTCNN()

# Détecter et recadrer le visage
img_cropped = mtcnn(img, save_path='hoda_cropped.png')

# Calculer l'embedding
img_embedding = resnet(img_cropped.unsqueeze(0))

In [43]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch

# Initialisation de MTCNN pour la détection faciale et InceptionResnetV1 pour les embeddings
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def compare_images(img_path_1, img_path_2, threshold=0.8):

    # Charger les deux images
    img1 = Image.open('yasmine_cropped.png')
    img2 = Image.open('hoda_cropped.png')
    
    # Détecter et recadrer les visages dans les deux images
    img1_cropped = mtcnn(img1)
    img2_cropped = mtcnn(img2)
    
    # Si un visage n'a pas été détecté, renvoyer une erreur
    if img1_cropped is None or img2_cropped is None:
        print("Erreur : visage non détecté dans l'une des images.")
        return False
    
    # Calculer les embeddings des deux images
    img1_embedding = resnet(img1_cropped.unsqueeze(0))
    img2_embedding = resnet(img2_cropped.unsqueeze(0))
    
    # Calculer la distance entre les deux embeddings (distance Euclidienne)
    distance = torch.dist(img1_embedding, img2_embedding).item()
    
    # Comparer la distance avec le seuil
    if distance < threshold:
        print(f"Les visages correspondent avec une distance de {distance:.2f}.")
        return True
    else:
        print(f"Les visages ne correspondent pas, distance : {distance:.2f}.")
        return False

# Exemple d'utilisation
img_captured = 'yasmine_cropped.png'
img_reference = 'hoda_cropped.png'
result = compare_images(img_captured, img_reference)

Les visages ne correspondent pas, distance : 1.28.
